### Dependencies

In [1]:
!git clone "https://github.com/suyash/ContextualDecomposition.git" && mv ContextualDecomposition/cd ./cd

Cloning into 'ContextualDecomposition'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 83 (delta 42), reused 82 (delta 41), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [2]:
!curl -L -o "job_dir.zip" "https://drive.google.com/uc?export=download&id=13Uyub6pPWWS9USmj2WxAilPMoYFQZtCd" && unzip -q -d "job_dir" "job_dir.zip"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    101      0 --:--:--  0:00:03 --:--:--   101
100 21.6M    0 21.6M    0     0  4141k      0 --:--:--  0:00:05 --:--:-- 29.5M


### Imports

In [3]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf

from cd.acd import lstm_acd_decomposition, agglomerate_acd_1d_decomposition
from cd.preprocess import create_table, create_inv_table

In [5]:
model = tf.keras.models.load_model("job_dir/saved_model/best")
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         1831808   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 1,963,650
Trainable params: 1,963,650
Non-trainable params: 0
_________________________________________________________________


In [0]:
tokens = np.load("job_dir/tokens.npy")
table = create_table(tokens)
inv_table = create_inv_table(tokens)

In [0]:
s = "it 's easy to love robin tunney -- she is pretty and she can act -- but it gets harder and harder to understand her choices ."

In [0]:
text = s.split()

In [9]:
inp = table.lookup(tf.constant(s.split()))
inp = tf.expand_dims(inp, 0)
inp

<tf.Tensor: shape=(1, 27), dtype=int32, numpy=
array([[  12,    8,  396,    7,   82, 3814, 5186,   28,  281,    9,  313,
           4,  281,   66,  590,   28,   22,   12,  283, 5449,    4, 5449,
           7,  683,  115, 2049,    6]], dtype=int32)>

In [10]:
comps, comp_scores = lstm_acd_decomposition(inp, model)

/content/cd/acd.py:29: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 99.5)
/content/cd/acd.py:15: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 59)
/content/cd/acd.py:21: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 59)
/content/cd/acd.py:24: RuntimeWarning: invalid value encountered in greater
  return a > np.nanpercentile(a, 49)


In [0]:
tree = agglomerate_acd_1d_decomposition(comps, comp_scores)

In [12]:
for i, l in enumerate(tree):
    print("Level", i)

    for s in l:
        print(" ".join(text[s[0][0]:s[0][1]]), ":", s[1])

    print("--------------------------------------------------------")

Level 0
it : 0.05622806400060654
's : -0.17336717247962952
easy : 0.1141328439116478
to : -0.27177995443344116
love : 1.7465598583221436
robin : 0.7639279365539551
tunney : 0.21042287349700928
-- : -0.02498093619942665
she : 0.7199404835700989
is : 0.2635386884212494
pretty : 1.263846755027771
and : 0.2790667414665222
she : 0.6174754500389099
can : 0.2957877516746521
act : 0.7715259790420532
-- : 0.10193504393100739
but : 0.7194859981536865
it : -0.014234669506549835
gets : -0.1873348355293274
harder : -3.147366523742676
and : 0.4087800979614258
harder : -3.0583136081695557
to : -0.0430406890809536
understand : -2.0601515769958496
her : 1.5029910802841187
choices : 1.3775818347930908
. : 0.3356570601463318
--------------------------------------------------------
Level 1
understand her : 1.0073017
love robin : 3.2940018
but it : -0.1526309
she is pretty : 2.6888027
--------------------------------------------------------
Level 2
to love robin : 1.9771428
understand her choices : 2.17721